In [72]:
# Data Manipulation and Preparation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Evaluation metrics and visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

import itertools
import time
import csv
import json
from IPython.display import Audio, display
import winsound
from threading import Thread

Using device: cuda


In [2]:
# Load cleaned dataset
df = pd.read_csv('BTC_1h_Complete_NoTA.csv', parse_dates=['open_time'])

In [4]:
# Convert to datetime 
df['open_time'] = pd.to_datetime(df['open_time'])
df['close_time'] = pd.to_datetime(df['close_time'])

In [39]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def directional_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true[1:] - y_true[:-1]) == np.sign(y_pred[1:] - y_pred[:-1]))


def plot_loss_over_epochs(training_loss, validation_loss, file_prefix, start_epoch=3):
    plt.figure(figsize=(10, 6))
    plt.plot(range(start_epoch, len(training_loss)), training_loss[start_epoch:], label='Training Loss')
    plt.plot(range(start_epoch, len(validation_loss)), validation_loss[start_epoch:], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.show()

In [40]:
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

def prepare_data(df, target_column, window_size, feature_columns):
    X = df[feature_columns].values
    y = df[target_column].values.reshape(-1, 1)

    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    return X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y

class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = nn.Tanh()
        self.fc2 = nn.Linear(dense_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

In [54]:
def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', use_early_stopping=True, plot_loss=False):
    # Extract parameters
    window_size = params['window_size']
    lstm_dim = params['lstm_dim']
    dense_dim = params['dense_dim']
    num_layers = params['num_layers']
    dropout = params['dropout']
    lr = params['lr']
    batch_size = params['batch_size']
    num_epochs = params['num_epochs']
    optimizer_type = params['optimizer_type']
    patience = params['patience']

    # Clear CUDA cache
    torch.cuda.empty_cache()

    # Prepare data
    X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, feature_columns)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
    y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # Model parameters
    input_dim = X_train_tensor.shape[2]
    output_dim = 1

    model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

    # Define loss and optimizer
    criterion = nn.MSELoss()
    optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

    # Training loop
    best_loss = float('inf')
    patience_counter = 0
    start_time = time.time()

    training_losses = []
    validation_losses = []

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()

        epoch_train_loss /= len(train_loader)
        training_losses.append(epoch_train_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()

        val_loss /= len(test_loader)
        validation_losses.append(val_loss)

        if use_early_stopping:
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                torch.save(model.state_dict(), f'{file_prefix}_{target_column}_best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break

    training_time = time.time() - start_time

    # Save the final model if early stopping was not used or not triggered
    if not use_early_stopping or patience_counter < patience:
        torch.save(model.state_dict(), f'{file_prefix}_{target_column}_final_model.pth')

    # Evaluate the model
    model.eval()
    evaluation_start_time = time.time()
    with torch.no_grad():
        train_predictions = model(X_train_tensor).cpu().numpy()
        test_predictions = model(X_test_tensor).cpu().numpy()

        # Inverse transform predictions and actual values
        y_train_seq = y_train_seq.reshape(-1, 1)
        y_test_seq = y_test_seq.reshape(-1, 1)
        train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
        test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
        y_train_inverse = scaler_y.inverse_transform(y_train_seq)
        y_test_inverse = scaler_y.inverse_transform(y_test_seq)

        train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
        test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
        train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
        test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
        train_rmse = np.sqrt(train_mse)
        test_rmse = np.sqrt(test_mse)
        train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
        test_r2 = r2_score(y_test_inverse, test_predictions_inverse)

        train_mape = np.mean(np.abs((y_train_inverse - train_predictions_inverse) / y_train_inverse)) * 100
        test_mape = np.mean(np.abs((y_test_inverse - test_predictions_inverse) / y_test_inverse)) * 100

        train_directional_acc = np.mean(np.sign(y_train_inverse[1:] - y_train_inverse[:-1]) == np.sign(train_predictions_inverse[1:] - train_predictions_inverse[:-1]))
        test_directional_acc = np.mean(np.sign(y_test_inverse[1:] - y_test_inverse[:-1]) == np.sign(test_predictions_inverse[1:] - test_predictions_inverse[:-1]))

    evaluation_time = time.time() - evaluation_start_time

    result = {
        "window_size": window_size,
        "lstm_dim": lstm_dim,
        "num_layers": num_layers,
        "dense_dim": dense_dim,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "optimizer_type": optimizer_type,
        "train_mse": train_mse,
        "test_mse": test_mse,
        "train_mae": train_mae,
        "test_mae": test_mae,
        "train_rmse": train_rmse,
        "test_rmse": test_rmse,
        "train_r2": train_r2,
        "test_r2": test_r2,
        "train_mape": train_mape,
        "test_mape": test_mape,
        "train_directional_acc": train_directional_acc,
        "test_directional_acc": test_directional_acc,
        "training_time": training_time,
        "evaluation_time": evaluation_time,
        "patience": patience
    }

    # Save result to a CSV file
    with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=result.keys())
        if f.tell() == 0:
            writer.writeheader()  # Write header only if file is empty
        writer.writerow(result)

    # Save training and validation losses to a JSON file
    losses = {
        "training_losses": training_losses,
        "validation_losses": validation_losses
    }
    with open(f'{file_prefix}_{target_column}_losses.json', 'w') as f:
        json.dump(losses, f)

    # Save predictions and actual values to a JSON file
    predictions = {
        "train_predictions": train_predictions_inverse.tolist(),
        "test_predictions": test_predictions_inverse.tolist(),
        "y_train": y_train_inverse.tolist(),
        "y_test": y_test_inverse.tolist()
    }
    with open(f'{file_prefix}_{target_column}_predictions.json', 'w') as f:
        json.dump(predictions, f)

    # Save hyperparameters to a JSON file, excluding any non-serializable types
    serializable_params = {k: v for k, v in params.items() if isinstance(v, (int, float, str, bool))}
    with open(f'{file_prefix}_{target_column}_params.json', 'w') as f:
        json.dump(serializable_params, f)

    # Print summary of the results
    print(f"Finished model with parameters: {params}")
    print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
    print(f"Train MAPE: {train_mape:.4f}, Test MAPE: {test_mape:.4f}")
    print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
    print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

    # Plot loss over epochs if specified
    if plot_loss:
        plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

    return result

In [55]:
def plot_saved_losses(file_prefix, target_column, start_epoch=3):
    # Load losses from JSON file
    with open(f'{file_prefix}_{target_column}_losses.json', 'r') as f:
        losses = json.load(f)
    
    training_losses = losses['training_losses']
    validation_losses = losses['validation_losses']
    
    # Plot losses
    plt.figure(figsize=(10, 6))
    plt.plot(range(start_epoch, len(training_losses)), training_losses[start_epoch:], label='Training Loss')
    plt.plot(range(start_epoch, len(validation_losses)), validation_losses[start_epoch:], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    # plt.savefig(f'{file_prefix}_loss_plot.png')
    plt.show()

# plot_saved_losses('results_complete', 'close')

def plot_predicted_vs_actual(file_prefix, target_column, dataset='test'):
    import json
    import matplotlib.pyplot as plt
    
    # Load predictions and actual values from JSON file
    with open(f'{file_prefix}_{target_column}_predictions.json', 'r') as f:
        data = json.load(f)
    
    if dataset == 'train':
        predictions = np.array(data['train_predictions'])
        actual = np.array(data['y_train'])
        title = 'Training Set: Predicted vs Actual'
    elif dataset == 'test':
        predictions = np.array(data['test_predictions'])
        actual = np.array(data['y_test'])
        title = 'Test Set: Predicted vs Actual'
    else:
        raise ValueError("Invalid dataset value. Use 'train' or 'test'.")
    
    # Plot predicted vs actual
    plt.figure(figsize=(10, 6))
    plt.plot(actual, label='Actual')
    plt.plot(predictions, label='Predicted')
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title(title)
    plt.legend()
    plt.savefig(f'{file_prefix}_{target_column}_predicted_vs_actual_{dataset}.png')
    plt.show()

# plot_predicted_vs_actual('results_complete', 'close', dataset='test')
# plot_predicted_vs_actual('results_complete', 'close', dataset='train')


In [56]:
# Define parameter space
parameter_space = {
    'window_size': [24],
    'lstm_dim': [96, 128, 150],
    'dense_dim': [96, 128, 150],
    'num_layers': [1],
    'dropout': [0.001, 0.0001],
    'lr': [0.0001, 0.001],
    'num_epochs': [50, 100, 150],
    'batch_size': [64],
    'optimizer_type': ['Adam'],
    'patience': [24],
    'activation_function': [nn.Tanh]
}

# Define target columns
target_columns = ['close']

# Define feature combinations
feature_combinations = [
    ['open', 'high', 'low', 'close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'open_p', 'high_p', 'low_p', 'close_p']
]


In [57]:
# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    for target_column in target_columns:
        for features in feature_combinations:
            print(f"Running model with parameters: {param_dict}, target column: {target_column}, features: {features}")
            # Run with early stopping
            train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete2', use_early_stopping=True, plot_loss=False)
            # Run with fixed epochs
            train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete_fixed_epochs2', use_early_stopping=False, plot_loss=False)

Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}, target column: close, features: ['open', 'high', 'low', 'close', 'volume', 'quote_vol', 'count', 'buy_base', 'buy_quote', 'open_p', 'high_p', 'low_p', 'close_p']


KeyboardInterrupt: 

# THE CODE ABOVE I SORT OF DELETED BY ACCIDENT I PRESSED GO AGAIN AND WIPED THE OUTPUT BUT IT SHOULD BE SAVED - HOPEFULLY NOTHING WAS OVERWRITTEN BUT I DONT THINK SO IT


In [ ]:
# the following has modified the variable management (cuda)
# also modified some of the printouts etc

In [51]:
# def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', use_early_stopping=True, plot_loss=False):
#     # Extract parameters
#     window_size = params['window_size']
#     lstm_dim = params['lstm_dim']
#     dense_dim = params['dense_dim']
#     num_layers = params['num_layers']
#     dropout = params['dropout']
#     lr = params['lr']
#     batch_size = params['batch_size']
#     num_epochs = params['num_epochs']
#     optimizer_type = params['optimizer_type']
#     patience = params['patience']

#     # Clear CUDA cache
#     torch.cuda.empty_cache()

#     # Prepare data
#     X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, feature_columns)

#     # Convert to PyTorch tensors
#     X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
#     X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
#     y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
#     y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

#     # Create data loaders
#     train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#     test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

#     # Clear intermediate data variables
#     del X_train_seq, X_test_seq, y_train_seq, y_test_seq
#     del X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor
#     del train_dataset, test_dataset
#     torch.cuda.empty_cache()

#     # Model parameters
#     input_dim = train_loader.dataset.tensors[0].shape[2]
#     output_dim = 1

#     model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

#     # Define loss and optimizer
#     criterion = nn.MSELoss()
#     optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

#     # Training loop
#     best_loss = float('inf')
#     patience_counter = 0
#     start_time = time.time()

#     training_losses = []
#     validation_losses = []

#     for epoch in range(num_epochs):
#         model.train()
#         epoch_train_loss = 0
#         for inputs, labels in train_loader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             epoch_train_loss += loss.item()
#             # Clear variables at the end of each iteration
#             del inputs, labels, outputs
#             torch.cuda.empty_cache()

#         epoch_train_loss /= len(train_loader)
#         training_losses.append(epoch_train_loss)

#         model.eval()
#         val_loss = 0
#         with torch.no_grad():
#             for inputs, labels in test_loader:
#                 outputs = model(inputs)
#                 val_loss += criterion(outputs, labels).item()
#                 # Clear variables at the end of each iteration
#                 del inputs, labels, outputs
#                 torch.cuda.empty_cache()

#         val_loss /= len(test_loader)
#         validation_losses.append(val_loss)

#         if use_early_stopping:
#             if val_loss < best_loss:
#                 best_loss = val_loss
#                 patience_counter = 0
#                 torch.save(model.state_dict(), f'{file_prefix}_{target_column}_best_model.pth')
#             else:
#                 patience_counter += 1
#                 if patience_counter >= patience:
#                     break

#     training_time = time.time() - start_time

#     # Save the final model if early stopping was not used or not triggered
#     if not use_early_stopping or patience_counter < patience:
#         torch.save(model.state_dict(), f'{file_prefix}_{target_column}_final_model.pth')

#     # Clear training-related variables
#     del train_loader, test_loader, criterion, optimizer
#     torch.cuda.empty_cache()

#     # Evaluate the model
#     model.eval()
#     evaluation_start_time = time.time()
#     with torch.no_grad():
#         train_predictions = model(train_loader.dataset.tensors[0]).cpu().numpy()
#         test_predictions = model(test_loader.dataset.tensors[0]).cpu().numpy()

#         # Inverse transform predictions and actual values
#         y_train_seq = train_loader.dataset.tensors[1].cpu().numpy()
#         y_test_seq = test_loader.dataset.tensors[1].cpu().numpy()
#         train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
#         test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
#         y_train_inverse = scaler_y.inverse_transform(y_train_seq)
#         y_test_inverse = scaler_y.inverse_transform(y_test_seq)

#         train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
#         test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
#         train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
#         test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
#         train_rmse = np.sqrt(train_mse)
#         test_rmse = np.sqrt(test_mse)
#         train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
#         test_r2 = r2_score(y_test_inverse, test_predictions_inverse)

#         train_mape = np.mean(np.abs((y_train_inverse - train_predictions_inverse) / y_train_inverse)) * 100
#         test_mape = np.mean(np.abs((y_test_inverse - test_predictions_inverse) / y_test_inverse)) * 100

#         train_directional_acc = np.mean(np.sign(y_train_inverse[1:] - y_train_inverse[:-1]) == np.sign(train_predictions_inverse[1:] - train_predictions_inverse[:-1]))
#         test_directional_acc = np.mean(np.sign(y_test_inverse[1:] - y_test_inverse[:-1]) == np.sign(test_predictions_inverse[1:] - test_predictions_inverse[:-1]))

#     evaluation_time = time.time() - evaluation_start_time

#     result = {
#         "window_size": window_size,
#         "lstm_dim": lstm_dim,
#         "num_layers": num_layers,
#         "dense_dim": dense_dim,
#         "dropout": dropout,
#         "lr": lr,
#         "batch_size": batch_size,
#         "num_epochs": num_epochs,
#         "optimizer_type": optimizer_type,
#         "train_mse": train_mse,
#         "test_mse": test_mse,
#         "train_mae": train_mae,
#         "test_mae": test_mae,
#         "train_rmse": train_rmse,
#         "test_rmse": test_rmse,
#         "train_r2": train_r2,
#         "test_r2": test_r2,
#         "train_mape": train_mape,
#         "test_mape": test_mape,
#         "train_directional_acc": train_directional_acc,
#         "test_directional_acc": test_directional_acc,
#         "training_time": training_time,
#         "evaluation_time": evaluation_time,
#         "patience": patience
#     }

#     # Save result to a CSV file
#     with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
#         writer = csv.DictWriter(f, fieldnames=result.keys())
#         if f.tell() == 0:
#             writer.writeheader()  # Write header only if file is empty
#         writer.writerow(result)

#     # Save training and validation losses to a JSON file
#     losses = {
#         "training_losses": training_losses,
#         "validation_losses": validation_losses
#     }
#     with open(f'{file_prefix}_{target_column}_losses.json', 'w') as f:
#         json.dump(losses, f)

#     # Save predictions and actual values to a JSON file
#     predictions = {
#         "train_predictions": train_predictions_inverse.tolist(),
#         "test_predictions": test_predictions_inverse.tolist(),
#         "y_train": y_train_inverse.tolist(),
#         "y_test": y_test_inverse.tolist()
#     }
#     with open(f'{file_prefix}_{target_column}_predictions.json', 'w') as f:
#         json.dump(predictions, f)

#     # Save hyperparameters to a JSON file, excluding any non-serializable types
#     serializable_params = {k: v for k, v in params.items() if isinstance(v, (int, float, str, bool))}
#     with open(f'{file_prefix}_{target_column}_params.json', 'w') as f:
#         json.dump(serializable_params, f)

#     # Clear evaluation-related variables
#     del train_predictions, test_predictions
#     del train_predictions_inverse, test_predictions_inverse
#     del y_train_inverse, y_test_inverse
#     del losses, predictions
#     torch.cuda.empty_cache()

#     # Print summary of the results
#     print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
#     print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
#     print(f"Train MAPE: {train_mape:.4f}, Test MAPE: {test_mape:.4f}")
#     print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
#     print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

#     # Plot loss over epochs if specified
#     if plot_loss:
#         plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

#     # Clear remaining variables
#     del model, result, serializable_params
#     torch.cuda.empty_cache()

#     return result

In [62]:
# def create_sequences(data, target, window_size):
#     sequences = []
#     labels = []
#     for i in range(len(data) - window_size):
#         seq = data[i:i + window_size]
#         label = target[i + window_size]
#         sequences.append(seq)
#         labels.append(label)
#     return np.array(sequences), np.array(labels)

# def prepare_data(df, target_column, window_size, feature_columns):
#     X = df[feature_columns].values
#     y = df[target_column].values.reshape(-1, 1)

#     scaler_X = MinMaxScaler()
#     X_scaled = scaler_X.fit_transform(X)

#     scaler_y = MinMaxScaler()
#     y_scaled = scaler_y.fit_transform(y)

#     X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
#     X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
#     X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

#     return X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y

# class LSTMModel(nn.Module):
#     def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
#         self.fc1 = nn.Linear(lstm_dim, dense_dim)
#         self.activation = nn.Tanh()
#         self.fc2 = nn.Linear(dense_dim, output_dim)

#     def forward(self, x):
#         x, _ = self.lstm(x)
#         x = x[:, -1, :]
#         x = self.fc1(x)
#         x = self.activation(x)
#         x = self.fc2(x)
#         return x

In [65]:
# def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', use_early_stopping=True, plot_loss=False):
#     result = {}  # Initialize result to ensure it's always defined
#     try:
#         # Extract parameters
#         window_size = params['window_size']
#         lstm_dim = params['lstm_dim']
#         dense_dim = params['dense_dim']
#         num_layers = params['num_layers']
#         dropout = params['dropout']
#         lr = params['lr']
#         batch_size = params['batch_size']
#         num_epochs = params['num_epochs']
#         optimizer_type = params['optimizer_type']
#         patience = params['patience']

#         # Clear CUDA cache
#         torch.cuda.empty_cache()

#         # Prepare data
#         X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, feature_columns)

#         # Convert to PyTorch tensors
#         X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
#         X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
#         y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
#         y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

#         # Create data loaders
#         train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
#         test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#         train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#         test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

#         # Model parameters
#         input_dim = X_train_tensor.shape[2]
#         output_dim = 1

#         model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

#         # Define loss and optimizer
#         criterion = nn.MSELoss()
#         optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

#         # Training loop
#         best_loss = float('inf')
#         patience_counter = 0
#         start_time = time.time()

#         training_losses = []
#         validation_losses = []

#         for epoch in range(num_epochs):
#             model.train()
#             epoch_train_loss = 0
#             for inputs, labels in train_loader:
#                 optimizer.zero_grad()
#                 outputs = model(inputs)
#                 loss = criterion(outputs, labels)
#                 loss.backward()
#                 optimizer.step()
#                 epoch_train_loss += loss.item()

#             epoch_train_loss /= len(train_loader)
#             training_losses.append(epoch_train_loss)

#             model.eval()
#             val_loss = 0
#             with torch.no_grad():
#                 for inputs, labels in test_loader:
#                     outputs = model(inputs)
#                     val_loss += criterion(outputs, labels).item()

#             val_loss /= len(test_loader)
#             validation_losses.append(val_loss)

#             if use_early_stopping:
#                 if val_loss < best_loss:
#                     best_loss = val_loss
#                     patience_counter = 0
#                     torch.save(model.state_dict(), f'{file_prefix}_{target_column}_best_model.pth')
#                 else:
#                     patience_counter += 1
#                     if patience_counter >= patience:
#                         break

#         training_time = time.time() - start_time

#         # Save the final model if early stopping was not used or not triggered
#         if not use_early_stopping or patience_counter < patience:
#             torch.save(model.state_dict(), f'{file_prefix}_{target_column}_final_model.pth')

#         # Evaluate the model
#         model.eval()
#         evaluation_start_time = time.time()
#         with torch.no_grad():
#             train_predictions = model(X_train_tensor).cpu().numpy()
#             test_predictions = model(X_test_tensor).cpu().numpy()

#             # Inverse transform predictions and actual values
#             y_train_seq = y_train_seq.reshape(-1, 1)
#             y_test_seq = y_test_seq.reshape(-1, 1)
#             train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
#             test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
#             y_train_inverse = scaler_y.inverse_transform(y_train_seq)
#             y_test_inverse = scaler_y.inverse_transform(y_test_seq)

#             train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
#             test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
#             train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
#             test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
#             train_rmse = np.sqrt(train_mse)
#             test_rmse = np.sqrt(test_mse)
#             train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
#             test_r2 = r2_score(y_test_inverse, test_predictions_inverse)

#             train_mape = np.mean(np.abs((y_train_inverse - train_predictions_inverse) / y_train_inverse)) * 100
#             test_mape = np.mean(np.abs((y_test_inverse - test_predictions_inverse) / y_test_inverse)) * 100

#             train_directional_acc = np.mean(np.sign(y_train_inverse[1:] - y_train_inverse[:-1]) == np.sign(train_predictions_inverse[1:] - train_predictions_inverse[:-1]))
#             test_directional_acc = np.mean(np.sign(y_test_inverse[1:] - y_test_inverse[:-1]) == np.sign(test_predictions_inverse[1:] - test_predictions_inverse[:-1]))

#         evaluation_time = time.time() - evaluation_start_time

#         result = {
#             "window_size": window_size,
#             "lstm_dim": lstm_dim,
#             "num_layers": num_layers,
#             "dense_dim": dense_dim,
#             "dropout": dropout,
#             "lr": lr,
#             "batch_size": batch_size,
#             "num_epochs": num_epochs,
#             "optimizer_type": optimizer_type,
#             "train_mse": train_mse,
#             "test_mse": test_mse,
#             "train_mae": train_mae,
#             "test_mae": test_mae,
#             "train_rmse": train_rmse,
#             "test_rmse": test_rmse,
#             "train_r2": train_r2,
#             "test_r2": test_r2,
#             "train_mape": train_mape,
#             "test_mape": test_mape,
#             "train_directional_acc": train_directional_acc,
#             "test_directional_acc": test_directional_acc,
#             "training_time": training_time,
#             "evaluation_time": evaluation_time,
#             "patience": patience
#         }

#         # Save result to a CSV file
#         with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
#             writer = csv.DictWriter(f, fieldnames=result.keys())
#             if f.tell() == 0:
#                 writer.writeheader()  # Write header only if file is empty
#             writer.writerow(result)

#         # Save training and validation losses to a JSON file
#         losses = {
#             "training_losses": training_losses,
#             "validation_losses": validation_losses
#         }
#         with open(f'{file_prefix}_{target_column}_losses.json', 'w') as f:
#             json.dump(losses, f)

#         # Save predictions and actual values to a JSON file
#         predictions = {
#             "train_predictions": train_predictions_inverse.tolist(),
#             "test_predictions": test_predictions_inverse.tolist(),
#             "y_train": y_train_inverse.tolist(),
#             "y_test": y_test_inverse.tolist()
#         }
#         with open(f'{file_prefix}_{target_column}_predictions.json', 'w') as f:
#             json.dump(predictions, f)

#     # Save hyperparameters to a JSON file, excluding any non-serializable types
#     serializable_params = {k: v for k, v in params.items() if isinstance(v, (int, float, str, bool))}
#     with open(f'{file_prefix}_{target_column}_params.json', 'w') as f:
#         json.dump(serializable_params, f)

#     # Print summary of the results
#     print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
#     print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
#     print(f"Train MAPE: {train_mape:.4f}, Test MAPE: {test_mape:.4f}")
#     print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
#     print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

#     # Plot loss over epochs if specified
#     if plot_loss:
#         plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

#     # Clear remaining variables
#     del model, result, serializable_params
#     torch.cuda.empty_cache()

#     # Plot loss over epochs if specified
#     if plot_loss:
#         plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

#     # Clear remaining variables
#     del model, result, serializable_params
#     torch.cuda.empty_cache()

#     return result
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         # Ensure result is always defined, even in case of error
#         result = {
#             "window_size": window_size,
#             "lstm_dim": lstm_dim,
#             "num_layers": num_layers,
#             "dense_dim": dense_dim,
#             "dropout": dropout,
#             "lr": lr,
#             "batch_size": batch_size,
#             "num_epochs": num_epochs,
#             "optimizer_type": optimizer_type,
#             "error": str(e)
#         }

#     return result

SyntaxError: expected 'except' or 'finally' block (1254552121.py, line 177)

In [89]:
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

def prepare_data(df, target_column, window_size, feature_columns):
    X = df[feature_columns].values
    y = df[target_column].values.reshape(-1, 1)

    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    return X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y

class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = nn.Tanh()
        self.fc2 = nn.Linear(dense_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', use_early_stopping=True, plot_loss=False):
    result = {}  # Initialize result to ensure it's always defined
    try:
        # Extract parameters
        window_size = params['window_size']
        lstm_dim = params['lstm_dim']
        dense_dim = params['dense_dim']
        num_layers = params['num_layers']
        dropout = params['dropout']
        lr = params['lr']
        batch_size = params['batch_size']
        num_epochs = params['num_epochs']
        optimizer_type = params['optimizer_type']
        patience = params['patience']

        # Clear CUDA cache
        torch.cuda.empty_cache()

        # Prepare data
        X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, feature_columns)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
        y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

        # Model parameters
        input_dim = X_train_tensor.shape[2]
        output_dim = 1

        model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

        # Define loss and optimizer
        criterion = nn.MSELoss()
        optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

        # Training loop
        best_loss = float('inf')
        patience_counter = 0
        start_time = time.time()

        training_losses = []
        validation_losses = []

        for epoch in range(num_epochs):
            model.train()
            epoch_train_loss = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            epoch_train_loss /= len(train_loader)
            training_losses.append(epoch_train_loss)

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()

            val_loss /= len(test_loader)
            validation_losses.append(val_loss)

            if use_early_stopping:
                if val_loss < best_loss:
                    best_loss = val_loss
                    patience_counter = 0
                    torch.save(model.state_dict(), f'{file_prefix}_{target_column}_best_model.pth')
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        break

        training_time = time.time() - start_time

        # Save the final model if early stopping was not used or not triggered
        if not use_early_stopping or patience_counter < patience:
            torch.save(model.state_dict(), f'{file_prefix}_{target_column}_final_model.pth')

        # Evaluate the model
        model.eval()
        evaluation_start_time = time.time()
        with torch.no_grad():
            train_predictions = model(X_train_tensor).cpu().numpy()
            test_predictions = model(X_test_tensor).cpu().numpy()

            # Inverse transform predictions and actual values
            y_train_seq = y_train_seq.reshape(-1, 1)
            y_test_seq = y_test_seq.reshape(-1, 1)
            train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
            test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
            y_train_inverse = scaler_y.inverse_transform(y_train_seq)
            y_test_inverse = scaler_y.inverse_transform(y_test_seq)

            train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
            test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
            train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
            test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
            train_rmse = np.sqrt(train_mse)
            test_rmse = np.sqrt(test_mse)
            train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
            test_r2 = r2_score(y_test_inverse, test_predictions_inverse)

            train_mape = np.mean(np.abs((y_train_inverse - train_predictions_inverse) / y_train_inverse)) * 100
            test_mape = np.mean(np.abs((y_test_inverse - test_predictions_inverse) / y_test_inverse)) * 100

            train_directional_acc = np.mean(np.sign(y_train_inverse[1:] - y_train_inverse[:-1]) == np.sign(train_predictions_inverse[1:] - train_predictions_inverse[:-1]))
            test_directional_acc = np.mean(np.sign(y_test_inverse[1:] - y_test_inverse[:-1]) == np.sign(test_predictions_inverse[1:] - test_predictions_inverse[:-1]))

        evaluation_time = time.time() - evaluation_start_time

        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "train_mse": train_mse,
            "test_mse": test_mse,
            "train_mae": train_mae,
            "test_mae": test_mae,
            "train_rmse": train_rmse,
            "test_rmse": test_rmse,
            "train_r2": train_r2,
            "test_r2": test_r2,
            "train_mape": train_mape,
            "test_mape": test_mape,
            "train_directional_acc": train_directional_acc,
            "test_directional_acc": test_directional_acc,
            "training_time": training_time,
            "evaluation_time": evaluation_time,
            "patience": patience
        }

        # Save result to a CSV file
        with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=result.keys())
            if f.tell() == 0:
                writer.writeheader()  # Write header only if file is empty
            writer.writerow(result)

        # Save training and validation losses to a JSON file
        losses = {
            "training_losses": training_losses,
            "validation_losses": validation_losses
        }
        with open(f'{file_prefix}_{target_column}_losses.json', 'w') as f:
            json.dump(losses, f)

        # Save predictions and actual values to a JSON file
        predictions = {
            "train_predictions": train_predictions_inverse.tolist(),
            "test_predictions": test_predictions_inverse.tolist(),
            "y_train": y_train_inverse.tolist(),
            "y_test": y_test_inverse.tolist()
        }
        with open(f'{file_prefix}_{target_column}_predictions.json', 'w') as f:
            json.dump(predictions, f)

        # Save hyperparameters to a JSON file, excluding any non-serializable types
        serializable_params = {k: v for k, v in params.items() if isinstance(v, (int, float, str, bool))}
        with open(f'{file_prefix}_{target_column}_params.json', 'w') as f:
            json.dump(serializable_params, f)

        # Print summary of the results
        print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
        print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
        print(f"Train MAPE: {train_mape:.4f}, Test MAPE: {test_mape:.4f}")
        print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
        print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

        # Plot loss over epochs if specified
        if plot_loss:
            plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure result is always defined, even in case of error
        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "error": str(e)
        }

    return result

In [67]:
# Define target column
target_column = ['log_close']

# Define feature combination
features = ['log_open', 'log_high', 'log_low', 'log_close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'open_p', 'high_p', 'low_p', 'close_p']


In [68]:
# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}\n")
    # Run with early stopping
    print('Results with early stopping:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete3', use_early_stopping=True, plot_loss=False)
    # Run with fixed epochs
    print('Results with fixed epochs:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete_fixed_epochs3', use_early_stopping=False, plot_loss=False)

Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results with early stopping:

Finished model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}
Results: Train MSE: 0.0001, Test MSE: 0.0001, Train MAE: 0.0072, Test MAE: 0.0060
Train RMSE: 0.0104, Test RMSE: 0.0075, Train R2: 0.9997, Test R2: 0.9995
Train MAPE: 0.0714, Test MAPE: 0.0564
Train Directional Accuracy: 0.4634, Test Directional Accuracy: 0.4646
Training Time: 51.1476 seconds, Evaluation Time: 0.1477 seconds

Results with fixed epochs:

Finished model with parameters: {'window_size': 24, 'lst

In [90]:
# Define feature combination
features = ['log_open', 'log_high', 'log_low', 'log_close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote']


In [91]:
# Start total timer
start = time.time()

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}\n")
    # Run with early stopping
    print('Results with early stopping:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete5', use_early_stopping=True, plot_loss=False)
    # Run with fixed epochs
    print('Results with fixed epochs:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete_fixed_epochs5', use_early_stopping=False, plot_loss=False)

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

# Function to play a beep sound in a loop
def play_beep():
    while not stop_beep_flag:
        winsound.Beep(2000, 600)
        time.sleep(0.7)
        
# Start the beep sound
stop_beep_flag = False
beep_thread = Thread(target=play_beep)
beep_thread.start()

input("Press Enter to stop the beep sound...")
stop_beep_flag = True
beep_thread.join()
print("Beep sound stopped.")

Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results with early stopping:

Results: Train MSE: 0.0001, Test MSE: 0.0000, Train MAE: 0.0058, Test MAE: 0.0037
Train RMSE: 0.0094, Test RMSE: 0.0057, Train R2: 0.9998, Test R2: 0.9997
Train MAPE: 0.0578, Test MAPE: 0.0346
Train Directional Accuracy: 0.4653, Test Directional Accuracy: 0.4656
Training Time: 52.2349 seconds, Evaluation Time: 0.1622 seconds

Results with fixed epochs:

Results: Train MSE: 0.0001, Test MSE: 0.0000, Train MAE: 0.0058, Test MAE: 0.0037
Train RMSE: 0.0093, Test RMSE: 0.0057, Train R2: 0.9998, Test R2: 0.9997
Train MAPE: 0.0574, Test MAPE: 0.0350
Train Directional Accuracy: 0.4666, Test Directional Accuracy: 0.4664
Training Time: 52.2403 seconds, Evaluation Time: 0.1437 seconds

Running

Press Enter to stop the beep sound... 


Beep sound stopped.


In [100]:
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

def prepare_data(df, target_column, window_size, feature_columns):
    X = df[feature_columns].values
    y = df[target_column].values.reshape(-1, 1)
    
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    return X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y

class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = nn.Tanh()
        self.fc2 = nn.Linear(dense_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', use_early_stopping=True, plot_loss=False):
    result = {}  # Initialize result to ensure it's always defined
    try:
        # Extract parameters
        window_size = params['window_size']
        lstm_dim = params['lstm_dim']
        dense_dim = params['dense_dim']
        num_layers = params['num_layers']
        dropout = params['dropout']
        lr = params['lr']
        batch_size = params['batch_size']
        num_epochs = params['num_epochs']
        optimizer_type = params['optimizer_type']
        patience = params['patience']

        # Clear CUDA cache
        torch.cuda.empty_cache()

        # Prepare data
        X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, feature_columns)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
        y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

        # Model parameters
        input_dim = X_train_tensor.shape[2]
        output_dim = 1

        model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

        # Define loss and optimizer
        criterion = nn.MSELoss()
        optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

        # Training loop
        best_loss = float('inf')
        patience_counter = 0
        start_time = time.time()

        training_losses = []
        validation_losses = []

        for epoch in range(num_epochs):
            model.train()
            epoch_train_loss = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            epoch_train_loss /= len(train_loader)
            training_losses.append(epoch_train_loss)

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()

            val_loss /= len(test_loader)
            validation_losses.append(val_loss)

            if use_early_stopping:
                if val_loss < best_loss:
                    best_loss = val_loss
                    patience_counter = 0
                    torch.save(model.state_dict(), f'{file_prefix}_{target_column}_best_model.pth')
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        break

        training_time = time.time() - start_time

        # Save the final model if early stopping was not used or not triggered
        if not use_early_stopping or patience_counter < patience:
            torch.save(model.state_dict(), f'{file_prefix}_{target_column}_final_model.pth')

        # Evaluate the model
        model.eval()
        evaluation_start_time = time.time()
        with torch.no_grad():
            train_predictions = model(X_train_tensor).cpu().numpy()
            test_predictions = model(X_test_tensor).cpu().numpy()

            # Inverse transform predictions and actual values
            y_train_seq = y_train_seq.reshape(-1, 1)
            y_test_seq = y_test_seq.reshape(-1, 1)
            train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
            test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
            y_train_inverse = scaler_y.inverse_transform(y_train_seq)
            y_test_inverse = scaler_y.inverse_transform(y_test_seq)

            # # Invert log transformation
            # train_predictions_inverse = np.exp(train_predictions_inverse) - 1
            # test_predictions_inverse = np.exp(test_predictions_inverse) - 1
            # y_train_inverse = np.exp(y_train_inverse) - 1
            # y_test_inverse = np.exp(y_test_inverse) - 1

            train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
            test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
            train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
            test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
            train_rmse = np.sqrt(train_mse)
            test_rmse = np.sqrt(test_mse)
            train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
            test_r2 = r2_score(y_test_inverse, test_predictions_inverse)

            train_mape = np.mean(np.abs((y_train_inverse - train_predictions_inverse) / y_train_inverse)) * 100
            test_mape = np.mean(np.abs((y_test_inverse - test_predictions_inverse) / y_test_inverse)) * 100

            train_directional_acc = np.mean(np.sign(y_train_inverse[1:] - y_train_inverse[:-1]) == np.sign(train_predictions_inverse[1:] - train_predictions_inverse[:-1]))
            test_directional_acc = np.mean(np.sign(y_test_inverse[1:] - y_test_inverse[:-1]) == np.sign(test_predictions_inverse[1:] - test_predictions_inverse[:-1]))

        evaluation_time = time.time() - evaluation_start_time

        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "train_mse": train_mse,
            "test_mse": test_mse,
            "train_mae": train_mae,
            "test_mae": test_mae,
            "train_rmse": train_rmse,
            "test_rmse": test_rmse,
            "train_r2": train_r2,
            "test_r2": test_r2,
            "train_mape": train_mape,
            "test_mape": test_mape,
            "train_directional_acc": train_directional_acc,
            "test_directional_acc": test_directional_acc,
            "training_time": training_time,
            "evaluation_time": evaluation_time,
            "patience": patience
        }

        # Save result to a CSV file
        with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=result.keys())
            if f.tell() == 0:
                writer.writeheader()  # Write header only if file is empty
            writer.writerow(result)

        # Save training and validation losses to a JSON file
        losses = {
            "training_losses": training_losses,
            "validation_losses": validation_losses
        }
        with open(f'{file_prefix}_{target_column}_losses.json', 'w') as f:
            json.dump(losses, f)

        # Save predictions and actual values to a JSON file
        predictions = {
            "train_predictions": train_predictions_inverse.tolist(),
            "test_predictions": test_predictions_inverse.tolist(),
            "y_train": y_train_inverse.tolist(),
            "y_test": y_test_inverse.tolist()
        }
        with open(f'{file_prefix}_{target_column}_predictions.json', 'w') as f:
            json.dump(predictions, f)

        # Save hyperparameters to a JSON file, excluding any non-serializable types
        serializable_params = {k: v for k, v in params.items() if isinstance(v, (int, float, str, bool))}
        with open(f'{file_prefix}_{target_column}_params.json', 'w') as f:
            json.dump(serializable_params, f)

        # Print summary of the results
        print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
        print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
        print(f"Train MAPE: {train_mape:.4f}, Test MAPE: {test_mape:.4f}")
        print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
        print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

        # Plot loss over epochs if specified
        if plot_loss:
            plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

    except Exception as e:
        print(f"An error occurred: {e}")
        # Ensure result is always defined, even in case of error
        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "error": str(e)
        }

    return result

In [94]:
# Start total timer
start = time.time()

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}\n")
    # Run with early stopping
    print('Results with early stopping:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete5', use_early_stopping=True, plot_loss=False)
    # Run with fixed epochs
    print('Results with fixed epochs:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete5_fixed', use_early_stopping=False, plot_loss=False)

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

# Function to play a beep sound in a loop
def play_beep():
    while not stop_beep_flag:
        winsound.Beep(2000, 600)
        time.sleep(0.7)
        
# Start the beep sound
stop_beep_flag = False
beep_thread = Thread(target=play_beep)
beep_thread.start()

input("Press Enter to stop the beep sound...")
stop_beep_flag = True
beep_thread.join()
print("Beep sound stopped.")


Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results with early stopping:

Results: Train MSE: 92285.7046, Test MSE: 81719.2967, Train MAE: 171.0536, Test MAE: 165.1405
Train RMSE: 303.7856, Test RMSE: 285.8659, Train R2: 0.9996, Test R2: 0.9996
Train MAPE: 0.5729, Test MAPE: 0.3653
Train Directional Accuracy: 0.4658, Test Directional Accuracy: 0.4679
Training Time: 49.9402 seconds, Evaluation Time: 0.1813 seconds

Results with fixed epochs:

Results: Train MSE: 98398.3183, Test MSE: 95692.2604, Train MAE: 178.2315, Test MAE: 188.2710
Train RMSE: 313.6851, Test RMSE: 309.3417, Train R2: 0.9996, Test R2: 0.9995
Train MAPE: 0.5911, Test MAPE: 0.4161
Train Directional Accuracy: 0.4627, Test Directional Accuracy: 0.4617
Training Time: 50.6274 seconds, Evaluati

Press Enter to stop the beep sound... 


Beep sound stopped.


In [95]:
# Define target column
target_column = ['log_close']

# Define feature combination
features = ['log_open', 'log_high', 'log_low', 'log_close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'open_p', 'high_p', 'low_p', 'close_p']


In [96]:
# Start total timer
start = time.time()

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}\n")
    # Run with early stopping
    print('Results with early stopping:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete6', use_early_stopping=True, plot_loss=False)
    # Run with fixed epochs
    print('Results with fixed epochs:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete6_(fixed)', use_early_stopping=False, plot_loss=False)

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

# Function to play a beep sound in a loop
def play_beep():
    while not stop_beep_flag:
        winsound.Beep(2000, 600)
        time.sleep(0.7)
        
# Start the beep sound
stop_beep_flag = False
beep_thread = Thread(target=play_beep)
beep_thread.start()


input("Press Enter to stop the beep sound...")
stop_beep_flag = True
beep_thread.join()
print("Beep sound stopped.")

Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results with early stopping:

Results: Train MSE: 105412.6889, Test MSE: 101252.8078, Train MAE: 192.3417, Test MAE: 203.8002
Train RMSE: 324.6732, Test RMSE: 318.2025, Train R2: 0.9996, Test R2: 0.9995
Train MAPE: 0.6307, Test MAPE: 0.4670
Train Directional Accuracy: 0.4652, Test Directional Accuracy: 0.4691
Training Time: 50.7363 seconds, Evaluation Time: 0.1807 seconds

Results with fixed epochs:

Results: Train MSE: 103390.7881, Test MSE: 122094.7894, Train MAE: 191.9788, Test MAE: 241.4375
Train RMSE: 321.5444, Test RMSE: 349.4206, Train R2: 0.9996, Test R2: 0.9994
Train MAPE: 0.6163, Test MAPE: 0.5464
Train Directional Accuracy: 0.4669, Test Directional Accuracy: 0.4684
Training Time: 50.3618 seconds, Eval

Press Enter to stop the beep sound... 


Beep sound stopped.


In [101]:
# Define target column
target_column = ['close']

# Define feature combination
features = ['open', 'high', 'low', 'close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote']

In [102]:
# Start total timer
start = time.time()

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}\n")
    # Run with early stopping
    print('Results with early stopping:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete7', use_early_stopping=True, plot_loss=False)
    # Run with fixed epochs
    print('Results with fixed epochs:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete7_(fixed)', use_early_stopping=False, plot_loss=False)

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results with early stopping:

Results: Train MSE: 76069.6537, Test MSE: 66389.5368, Train MAE: 158.5131, Test MAE: 144.2361
Train RMSE: 275.8073, Test RMSE: 257.6617, Train R2: 0.9997, Test R2: 0.9997
Train MAPE: 0.5636, Test MAPE: 0.3188
Train Directional Accuracy: 0.4670, Test Directional Accuracy: 0.4716
Training Time: 49.9024 seconds, Evaluation Time: 0.1472 seconds

Results with fixed epochs:

Results: Train MSE: 75315.5705, Test MSE: 67412.9219, Train MAE: 157.7378, Test MAE: 144.9901
Train RMSE: 274.4368, Test RMSE: 259.6400, Train R2: 0.9997, Test R2: 0.9997
Train MAPE: 0.5689, Test MAPE: 0.3191
Train Directional Accuracy: 0.4676, Test Directional Accuracy: 0.4732
Training Time: 49.1868 seconds, Evaluati

In [103]:
# Define target column
target_column = ['close']

# Define feature combination
features = ['open', 'high', 'low', 'close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'open_p', 'high_p', 'low_p', 'close_p']

In [104]:
# Start total timer
start = time.time()

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}\n")
    # Run with early stopping
    print('Results with early stopping:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete8', use_early_stopping=True, plot_loss=False)
    # Run with fixed epochs
    print('Results with fixed epochs:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results_complete8_(fixed)', use_early_stopping=False, plot_loss=False)

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

# Start the beep sound
stop_beep_flag = False
beep_thread = Thread(target=play_beep)
beep_thread.start()

input("Press Enter to stop the beep sound...")
stop_beep_flag = True
beep_thread.join()
print("Beep sound stopped.")

Running model with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results with early stopping:

Results: Train MSE: 74026.1611, Test MSE: 66832.4586, Train MAE: 156.7481, Test MAE: 146.9144
Train RMSE: 272.0775, Test RMSE: 258.5197, Train R2: 0.9997, Test R2: 0.9997
Train MAPE: 0.5572, Test MAPE: 0.3288
Train Directional Accuracy: 0.4698, Test Directional Accuracy: 0.4701
Training Time: 52.3403 seconds, Evaluation Time: 0.1431 seconds

Results with fixed epochs:

Results: Train MSE: 81760.4684, Test MSE: 72530.5467, Train MAE: 164.9786, Test MAE: 154.3134
Train RMSE: 285.9379, Test RMSE: 269.3150, Train R2: 0.9997, Test R2: 0.9997
Train MAPE: 0.5706, Test MAPE: 0.3445
Train Directional Accuracy: 0.4708, Test Directional Accuracy: 0.4767
Training Time: 50.6061 seconds, Evaluati

Press Enter to stop the beep sound... 


Beep sound stopped.


In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined